In [1]:
#OPAN5510 Lab Assignment - Joins

#This lab focuses on using Polars to perform data joins and aggregations to answer business questions.

#Bike Trips Dataset

#Prerequisites

#For this assignment, you'll need to use Polars for data manipulation.

#Insert a code block to import necessary packages (polars)

In [5]:
import polars as pl

#Load Bike Trips Data Set

trips = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_trips.csv", null_values='NA')
weather = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/bike_weather.csv", null_values=['NA',''])

trips = trips.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date")) # ensure date column is typed correctly
weather = weather.with_columns(pl.col("date").str.strptime(pl.Date, "%Y-%m-%d").alias("date")) # ensure date column is typed correctly
print(trips.height)
print(weather.height)

78704
733


In [6]:
trips.head()

id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,date
i64,i64,str,str,i64,str,str,i64,i64,str,i64,date
4721,3,"""2013-08-29T20:27:00Z""","""Market at 10th""",67,"""2013-08-29T20:30:00Z""","""South Van Ness at Market""",66,416,"""Subscriber""",94107,2013-08-29
4812,3,"""2013-08-29T21:30:00Z""","""2nd at Folsom""",62,"""2013-08-29T21:33:00Z""","""2nd at Folsom""",62,409,"""Subscriber""",94107,2013-08-29
4705,3,"""2013-08-29T20:15:00Z""","""Golden Gate at Polk""",59,"""2013-08-29T20:18:00Z""","""San Francisco City Hall""",58,519,"""Subscriber""",94107,2013-08-29
4841,3,"""2013-08-29T21:48:00Z""","""University and Emerson""",35,"""2013-08-29T21:51:00Z""","""Cowper at University""",37,83,"""Subscriber""",94107,2013-08-29
4668,4,"""2013-08-29T19:39:00Z""","""San Francisco Caltrain 2 (330 …",69,"""2013-08-29T19:43:00Z""","""Townsend at 7th""",65,489,"""Subscriber""",94107,2013-08-29


In [7]:
weather.head()

date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,max_sea_level_pressure_inches,mean_sea_level_pressure_inches,min_sea_level_pressure_inches,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
date,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,i64
2013-08-29,74,68,61,61,58,56,93,75,57,30.07,30.02,29.97,10,10,10,23,11,28,"""0""",4,null,286,94107
2013-08-30,78,69,60,61,58,56,90,70,50,30.05,30.0,29.93,10,10,7,29,13,35,"""0""",2,null,291,94107
2013-08-31,71,64,57,57,56,54,93,75,57,30.0,29.96,29.92,10,10,10,26,15,31,"""0""",4,null,284,94107
2013-09-01,74,66,58,60,56,53,87,68,49,29.96,29.93,29.91,10,10,10,25,13,29,"""0""",4,null,284,94107
2013-09-02,75,69,62,61,60,58,93,77,61,29.97,29.94,29.9,10,10,6,23,12,30,"""0""",6,null,277,94107


In [8]:
# Question 1 - PART A

trips_weather = trips.join(weather, on='date', how='left')
trips_weather.head()

id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,max_sea_level_pressure_inches,mean_sea_level_pressure_inches,min_sea_level_pressure_inches,max_visibility_miles,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code_right
i64,i64,str,str,i64,str,str,i64,i64,str,i64,date,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,i64,str,i64,str,i64,i64
4721,3,"""2013-08-29T20:27:00Z""","""Market at 10th""",67,"""2013-08-29T20:30:00Z""","""South Van Ness at Market""",66,416,"""Subscriber""",94107,2013-08-29,74,68,61,61,58,56,93,75,57,30.07,30.02,29.97,10,10,10,23,11,28,"""0""",4,null,286,94107
4812,3,"""2013-08-29T21:30:00Z""","""2nd at Folsom""",62,"""2013-08-29T21:33:00Z""","""2nd at Folsom""",62,409,"""Subscriber""",94107,2013-08-29,74,68,61,61,58,56,93,75,57,30.07,30.02,29.97,10,10,10,23,11,28,"""0""",4,null,286,94107
4705,3,"""2013-08-29T20:15:00Z""","""Golden Gate at Polk""",59,"""2013-08-29T20:18:00Z""","""San Francisco City Hall""",58,519,"""Subscriber""",94107,2013-08-29,74,68,61,61,58,56,93,75,57,30.07,30.02,29.97,10,10,10,23,11,28,"""0""",4,null,286,94107
4841,3,"""2013-08-29T21:48:00Z""","""University and Emerson""",35,"""2013-08-29T21:51:00Z""","""Cowper at University""",37,83,"""Subscriber""",94107,2013-08-29,74,68,61,61,58,56,93,75,57,30.07,30.02,29.97,10,10,10,23,11,28,"""0""",4,null,286,94107
4668,4,"""2013-08-29T19:39:00Z""","""San Francisco Caltrain 2 (330 …",69,"""2013-08-29T19:43:00Z""","""Townsend at 7th""",65,489,"""Subscriber""",94107,2013-08-29,74,68,61,61,58,56,93,75,57,30.07,30.02,29.97,10,10,10,23,11,28,"""0""",4,null,286,94107


In [18]:
# Question 1 - PART B

avg_bad_weather = (
    trips_weather.filter(pl.col("events").is_not_null())
    .select(avg_duration = pl.col("duration").mean()).item()
)
print(f"the average trip duration during bad weather was {avg_bad_weather:.2f} minutes")

the average trip duration during bad weather was 10.35 minutes


In [31]:
# Load Batting and People DataFrame

batting = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/Batting.csv", null_values='NA')
people = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/People.csv", null_values='NA')
hallOfFame = pl.read_csv("https://raw.githubusercontent.com/philhetzel/opan5510-class9/refs/heads/main/data/HallOfFame.csv", null_values='NA')

In [26]:
batting.head()

playerID,yearID,stint,teamID,lgID,G,AB,R,H,X2B,X3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,i64
"""abercda01""",1871,1,"""TRO""",null,1,4,0,0,0,0,0,0,0,0,0,0,null,null,null,null,0
"""addybo01""",1871,1,"""RC1""",null,25,118,30,32,6,0,0,13,8,1,4,0,null,null,null,null,0
"""allisar01""",1871,1,"""CL1""",null,29,137,28,40,4,5,0,19,3,1,2,5,null,null,null,null,1
"""allisdo01""",1871,1,"""WS3""",null,27,133,28,44,10,2,2,27,1,1,0,2,null,null,null,null,0
"""ansonca01""",1871,1,"""RC1""",null,25,120,29,39,11,3,0,16,6,2,2,1,null,null,null,null,0


In [28]:
people.head()

playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,deathDate,birthDate
str,i64,i64,i64,str,str,str,i64,i64,i64,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str
"""aardsda01""",1981,12,27,"""USA""","""CO""","""Denver""",null,null,null,null,null,null,"""David""","""Aardsma""","""David Allan""",215,75,"""R""","""R""","""2004-04-06""","""2015-08-23""","""aardd001""","""aardsda01""",null,"""1981-12-27"""
"""aaronha01""",1934,2,5,"""USA""","""AL""","""Mobile""",2021,1,22,"""USA""","""GA""","""Atlanta""","""Hank""","""Aaron""","""Henry Louis""",180,72,"""R""","""R""","""1954-04-13""","""1976-10-03""","""aaroh101""","""aaronha01""","""2021-01-22""","""1934-02-05"""
"""aaronto01""",1939,8,5,"""USA""","""AL""","""Mobile""",1984,8,16,"""USA""","""GA""","""Atlanta""","""Tommie""","""Aaron""","""Tommie Lee""",190,75,"""R""","""R""","""1962-04-10""","""1971-09-26""","""aarot101""","""aaronto01""","""1984-08-16""","""1939-08-05"""
"""aasedo01""",1954,9,8,"""USA""","""CA""","""Orange""",null,null,null,null,null,null,"""Don""","""Aase""","""Donald William""",190,75,"""R""","""R""","""1977-07-26""","""1990-10-03""","""aased001""","""aasedo01""",null,"""1954-09-08"""
"""abadan01""",1972,8,25,"""USA""","""FL""","""Palm Beach""",null,null,null,null,null,null,"""Andy""","""Abad""","""Fausto Andres""",184,73,"""L""","""L""","""2001-09-10""","""2006-04-13""","""abada001""","""abadan01""",null,"""1972-08-25"""


In [29]:
hallOfFame.head()

playerID,yearID,votedBy,ballots,needed,votes,inducted,category,needed_note
str,i64,str,i64,i64,i64,str,str,str
"""cobbty01""",1936,"""BBWAA""",226,170,222,"""Y""","""Player""",null
"""ruthba01""",1936,"""BBWAA""",226,170,215,"""Y""","""Player""",null
"""wagneho01""",1936,"""BBWAA""",226,170,215,"""Y""","""Player""",null
"""mathech01""",1936,"""BBWAA""",226,170,205,"""Y""","""Player""",null
"""johnswa01""",1936,"""BBWAA""",226,170,189,"""Y""","""Player""",null


In [32]:
# Question 2 - PART A

stats_w_bio = batting.join(people, on='playerID', how='left')
stats_w_bio.head()

playerID,yearID,stint,teamID,lgID,G,AB,R,H,X2B,X3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,deathCountry,deathState,deathCity,nameFirst,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID,deathDate,birthDate
str,i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,i64,i64,i64,i64,str,str,str,i64,i64,i64,str,str,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str
"""abercda01""",1871,1,"""TRO""",null,1,4,0,0,0,0,0,0,0,0,0,0,null,null,null,null,0,1850,1,2,"""USA""","""OK""","""Fort Towson""",1939,11,11,"""USA""","""PA""","""Philadelphia""","""Frank""","""Abercrombie""","""Francis Patterson""",null,null,null,null,"""1871-10-21""","""1871-10-21""","""aberd101""","""abercda01""","""1939-11-11""","""1850-01-02"""
"""addybo01""",1871,1,"""RC1""",null,25,118,30,32,6,0,0,13,8,1,4,0,null,null,null,null,0,1842,2,null,"""CAN""","""ON""","""Port Hope""",1910,4,9,"""USA""","""ID""","""Pocatello""","""Bob""","""Addy""","""Robert Edward""",160,68,"""L""","""L""","""1871-05-06""","""1877-10-06""","""addyb101""","""addybo01""","""1910-04-09""",null
"""allisar01""",1871,1,"""CL1""",null,29,137,28,40,4,5,0,19,3,1,2,5,null,null,null,null,1,1849,1,29,"""USA""","""PA""","""Philadelphia""",1916,2,25,"""USA""","""DC""","""Washington""","""Art""","""Allison""","""Arthur Algernon""",150,68,null,null,"""1871-05-04""","""1876-10-05""","""allia101""","""allisar01""","""1916-02-25""","""1849-01-29"""
"""allisdo01""",1871,1,"""WS3""",null,27,133,28,44,10,2,2,27,1,1,0,2,null,null,null,null,0,1846,7,12,"""USA""","""PA""","""Philadelphia""",1916,12,19,"""USA""","""DC""","""Washington""","""Doug""","""Allison""","""Douglas L.""",160,70,"""R""","""R""","""1871-05-05""","""1883-07-13""","""allid101""","""allisdo01""","""1916-12-19""","""1846-07-12"""
"""ansonca01""",1871,1,"""RC1""",null,25,120,29,39,11,3,0,16,6,2,2,1,null,null,null,null,0,1852,4,17,"""USA""","""IA""","""Marshalltown""",1922,4,14,"""USA""","""IL""","""Chicago""","""Cap""","""Anson""","""Adrian Constantine""",227,72,"""R""","""R""","""1871-05-06""","""1897-10-03""","""ansoc101""","""ansonca01""","""1922-04-14""","""1852-04-17"""


In [51]:
# Question 2 - PART B

florida_hr = (
    stats_w_bio.filter(pl.col("birthState")=="FL")
    .select(total_hr = pl.col("HR").sum()).item()
)

print(florida_hr)

16225


In [72]:
# Question 3 - PART A

hall_inducted = (
    HallOfFame.filter((pl.col("inducted")=="Y") &  (pl.col("category")=="Player")).select("playerID").unique()
)
hall_inducted.head()

playerID
str
"""kleinch01"""
"""simmote01"""
"""mendejo99"""
"""griffke02"""
"""ansonca01"""


In [77]:
# Question 3 - PART B

hof_stats = batting.join(hall_inducted, on='playerID', how='inner')
hof_stats.head()

hof_hits = (
    hof_stats
    .group_by("playerID")
    .agg(pl.sum("H").alias("total_hits"))
    .select(pl.mean("total_hits").alias("average_hits"))
)

hof_hits.head()

average_hits
f64
1717.29918
